# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import csv

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [101]:
#simple calculations
school_unique = school_data_complete["school_name"].nunique()
students_total = school_data_complete["student_name"].count()
budget_total = school_data["budget"].sum()
math_avg = school_data_complete["math_score"].mean()
reading_avg = school_data_complete["reading_score"].mean()

#conditional calculations
read = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
read_list = read.count()
read_pass = read_list.max()
read_perc = read_pass / students_total * 100
math = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
math_list = math.count()
math_pass = math_list.max() 
math_perc = math_pass / students_total * 100
school_data_complete["Total"] = school_data_complete["math_score"] + school_data_complete["reading_score"]
total = school_data_complete.loc[school_data_complete["Total"] >= 140, :]
total_list = total.count()
total_pass = total_list.max()
total_perc = total_pass / students_total * 100

#create data frame and format
summary = pd.DataFrame({"Total Schools": [school_unique], "Total Students": [students_total], "Total Budget": [budget_total],
                       "% Passing Math": math_perc, "% Passing Reading": read_perc, "% Overall Passing Rate": total_perc})
summary["Total Students"] = summary["Total Students"].map("{:,}".format)
summary["Total Budget"] = summary["Total Budget"].map("${:,}".format)
summary["% Passing Math"] = summary["% Passing Math"].map("{:.2f}%".format)
summary["% Passing Reading"] = summary["% Passing Reading"].map("{:.2f}%".format)
summary["% Overall Passing Rate"] = summary["% Overall Passing Rate"].map("{:.2f}%".format)
summary


,Total Schools,Total Students,Total Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",74.98%,85.81%,89.39%


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [97]:
#seeing if this can be done with bins
bins = [0,69,100]
labels = [0, 1]
school_data_complete["Math Pass"] = pd.cut(school_data_complete["math_score"], bins, labels=labels)
school_data_complete["Reading Pass"] = pd.cut(school_data_complete["reading_score"], bins, labels=labels)

binstotal = [0,139,200]
school_data_complete["Total Pass"] = pd.cut(school_data_complete["Total"], binstotal, labels=labels)

#labels spit out as strings, need to convert to numeric
school_data_complete["Math Pass"] = school_data_complete["Math Pass"].astype("int64")
school_data_complete["Reading Pass"] = school_data_complete["Reading Pass"].astype("int64")
school_data_complete["Total Pass"] = school_data_complete["Total Pass"].astype("int64")

#group by school name and calculate values
group = school_data_complete.groupby(["school_name"])

name = group["school_name"].unique()
students = group["student_name"].count()
type_ = group["type"].max()
budget = group["budget"].max()
per_budget = group["budget"].max() / group["student_name"].count()
math = group["math_score"].mean()
read = group["reading_score"].mean()
math_perc = group["Math Pass"].sum() / group["student_name"].count() * 100
read_perc = group["Reading Pass"].sum() / group["student_name"].count() * 100
total_perc = (math_perc + read_perc) / 2

#create data frame and format
school = pd.DataFrame({"School Type": type_, "Total Students": students, "Total School Budget":budget, 
                       "Average Math Score": math, "Average Reading Score": read, "% Passing Math": math_perc, 
                       "% Passing Reading": read_perc, "% Overall Passing Rate": total_perc})

school["Total Students"] = school["Total Students"].map("{:,}".format)
school["Total School Budget"] = school["Total School Budget"].map("${:,}".format)
school["% Passing Math"] = school["% Passing Math"].map("{:.2f}%".format)
school["% Passing Reading"] = school["% Passing Reading"].map("{:.2f}%".format)
school["% Overall Passing Rate"] = school["% Overall Passing Rate"].map("{:.2f}%".format)
school["Average Math Score"] = school["Average Math Score"].map("{:.2f}".format)
school["Average Reading Score"] = school["Average Reading Score"].map("{:.2f}".format)

#sort by overall passing rate
school_sort = school.sort_values("% Overall Passing Rate", ascending=False)
school_sort.head()

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130",83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,"1,468","$917,500",83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858",83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,"2,283","$1,319,574",83.27,83.99,93.87%,96.54%,95.20%



## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [83]:
school_low = school.sort_values("% Overall Passing Rate", ascending=True)

school_low.head()

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",76.84%,80.74%,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411",76.71%,81.16%,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635",76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",77.07%,80.97%,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916",77.10%,80.75%,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [78]:
#use pd.pivot_table function to group the data
pivot = pd.pivot_table(school_data_complete,index=["school_name"],values=["math_score"],columns=["grade"],aggfunc=[np.mean])

#remove the index name
pivot = pivot.rename_axis(None)

#remove the multilevel index name and reorder the columns
pivot.columns = ["9th", "10th", "11th", "12th"]
pivot["9th"] = pivot["9th"].map("{:.2f}".format)
pivot["10th"] = pivot["10th"].map("{:.2f}".format)
pivot["11th"] = pivot["11th"].map("{:.2f}".format)
pivot["12th"] = pivot["12th"].map("{:.2f}".format)
pivot


,9th,10th,11th,12th
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04
Hernandez High School,77.34,77.14,77.19,77.44
Holden High School,83.43,85.00,82.86,83.79
Huang High School,75.91,76.45,77.23,77.03
Johnson High School,76.69,77.49,76.86,77.19
Pena High School,83.37,84.33,84.12,83.63


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [79]:
#same as above, just change the values[]
pivot = pd.pivot_table(school_data_complete,index=["school_name"],values=["reading_score"],columns=["grade"],aggfunc=[np.mean])
pivot = pivot.rename_axis(None)
pivot.columns = ["9th", "10th", "11th", "12th"]
pivot["9th"] = pivot["9th"].map("{:.2f}".format)
pivot["10th"] = pivot["10th"].map("{:.2f}".format)
pivot["11th"] = pivot["11th"].map("{:.2f}".format)
pivot["12th"] = pivot["12th"].map("{:.2f}".format)
pivot



,9th,10th,11th,12th
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37
Hernandez High School,80.66,81.40,80.86,80.87
Holden High School,83.32,83.82,84.70,83.68
Huang High School,81.51,81.42,80.31,81.29
Johnson High School,80.77,80.62,81.23,81.26
Pena High School,83.61,84.34,84.59,83.81


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [96]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_data_complete["per_student_spend"] = school_data_complete["budget"] / school_data_complete["size"]
school_data_complete["Spending Ranges (Per Student)"] = pd.cut(school_data_complete["per_student_spend"], spending_bins, labels=group_names)

#group by school name and calculate values
group = school_data_complete.groupby(["Spending Ranges (Per Student)"])

spend_range = group["Spending Ranges (Per Student)"].unique()
math = group["math_score"].mean()
read = group["reading_score"].mean()
math_perc = group["Math Pass"].sum() / group["student_name"].count() * 100
read_perc = group["Reading Pass"].sum() / group["student_name"].count() * 100
total_perc = (math_perc + read_perc) / 2

#create the data frame and format
spend_summary = pd.DataFrame({"Average Math Score":math, "Average Reading Score":read, 
                              "% Passing Math":math_perc, "% Passing Reading":read_perc, "% Overall Passing Rate":total_perc})

spend_summary["% Passing Math"] = spend_summary["% Passing Math"].map("{:.2f}%".format)
spend_summary["% Passing Reading"] = spend_summary["% Passing Reading"].map("{:.2f}%".format)
spend_summary["% Overall Passing Rate"] = spend_summary["% Overall Passing Rate"].map("{:.2f}%".format)
spend_summary["Average Math Score"] = spend_summary["Average Math Score"].map("{:.2f}".format)
spend_summary["Average Reading Score"] = spend_summary["Average Reading Score"].map("{:.2f}".format)

spend_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,93.70%,96.69%,95.19%
$585-615,83.53,83.84,94.12%,95.89%,95.01%
$615-645,78.06,81.43,71.40%,83.61%,77.51%
$645-675,77.05,81.01,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [99]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_data_complete["School Size"] = pd.cut(school_data_complete["size"], size_bins, labels=group_names)

#group by school size and calculate values
group = school_data_complete.groupby(["School Size"])

math = group["math_score"].mean()
read = group["reading_score"].mean()
math_perc = group["Math Pass"].sum() / group["student_name"].count() * 100
read_perc = group["Reading Pass"].sum() / group["student_name"].count() * 100
total_perc = (math_perc + read_perc) / 2

#create the data frame and format
size_summary = pd.DataFrame({"Average Math Score":math, "Average Reading Score":read, 
                              "% Passing Math":math_perc, "% Passing Reading":read_perc, "% Overall Passing Rate":total_perc})

size_summary["% Passing Math"] = size_summary["% Passing Math"].map("{:.2f}%".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:.2f}%".format)
size_summary["% Overall Passing Rate"] = size_summary["% Overall Passing Rate"].map("{:.2f}%".format)
size_summary["Average Math Score"] = size_summary["Average Math Score"].map("{:.2f}".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:.2f}".format)

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,95.00%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,95.19%
Large (2000-5000),77.48,81.20,68.65%,82.13%,75.39%


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [102]:
#group by school type
group = school_data_complete.groupby(["type"])

#calculate values
math = group["math_score"].mean()
read = group["reading_score"].mean()
math_perc = group["Math Pass"].sum() / group["student_name"].count() * 100
read_perc = group["Reading Pass"].sum() / group["student_name"].count() * 100
total_perc = (math_perc + read_perc) / 2

#create the data frame and format
type_summary = pd.DataFrame({"Average Math Score":math, "Average Reading Score":read, 
                              "% Passing Math":math_perc, "% Passing Reading":read_perc, "% Overall Passing Rate":total_perc})

type_summary["% Passing Math"] = type_summary["% Passing Math"].map("{:.2f}%".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:.2f}%".format)
type_summary["% Overall Passing Rate"] = type_summary["% Overall Passing Rate"].map("{:.2f}%".format)
type_summary["Average Math Score"] = type_summary["Average Math Score"].map("{:.2f}".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:.2f}".format)

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
